In [1]:
import pandas as pd
import os

In [2]:
def change_names(root_folder:str):
    for dir in os.listdir(root_folder):
        if not dir.startswith('.'):
            party_folder = root_folder + dir + '/'
            print(party_folder)
            for im_folder in os.listdir(party_folder):
                if not im_folder.startswith('.') and im_folder.startswith('im'):
                    im_folder = party_folder + im_folder + '/'
                    counter = 0
                    for image_name in os.listdir(im_folder):
                        old_name = im_folder + image_name
                        _, extension = os.path.splitext(image_name)
                        new_name = im_folder + dir + str(counter)+ extension
                        os.rename(old_name, new_name)
                        counter += 1

In [ ]:
# Change format from many to only jpeg

root_folder = '/Users/pauor506/Documents/CDHU/June2022:Dec2022/Pilots/Youtube_Katrin/Data_party_logos/'


for dir in os.listdir(root_folder):
    if not dir.startswith('.'):
        party_folder = root_folder + dir + '/'
        print(party_folder)
        for image in os.listdir(party_folder+'images'):
            if not image.startswith('.') and not image.endswith('.jpeg'):
                os.replace(party_folder+'images/'+image, party_folder+'images/'+image.split('.')[0]+'.jpeg')
                #print(party_folder+'images/'+image)
                #print(party_folder+'images/'+image.split('.')[0]+'.jpeg')


In [ ]:
root_folder = '/Users/pauor506/Documents/CDHU/June2022:Dec2022/Pilots/Youtube_Katrin/Data_party_logos'
party = 'España_Izquierda_unida_partido'
party_folder = root_folder + '/' + party

labels_list = [label.split('.')[0] for label in os.listdir(party_folder+'/'+'labels')]
imas_list = [label.split('.')[0] for label in os.listdir(party_folder+'/'+'images')]
print(f'lenghts are: {len(labels_list)} for labels and {len(imas_list)} for images')
match_list = set(labels_list) & set(imas_list)
print(f'Match lenght: {len(match_list)}')


In [ ]:
for im_name in match_list:
    os.replace(party_folder+'/'+'images'+'/'+im_name+'.jpeg', party_folder+'/'+'images_with_label'+'/'+im_name+'.jpeg')

### Data set split train and test 

This section is desinged to process the data from LabelMe into Yolo format. The flow is like follows:
1. Transform every party folder with json files to YOLO (txt) format files (this has the downside that it assigns label 0 to all classes; do not mix parties). 
2. Split images and label folders into train, validation and testing subfolders. 
3. Re-label the YOLO files so that of there are two parties, then one partie has label 0 and the other label 1. 

In [3]:
import fileinput
import os
import shutil
import random

In [4]:
# Function to transform the labels from Labelme format to YOLO 
# It takes a folder with json files and outputs another folder called "labels_txt" with txt files
def jsonTOyolo(main_folder:str): 
    # Set the main folder path
    #main_folder = "/Users/pauor506/Documents/CDHU/June2022:Dec2022/Pilots/Youtube_Katrin/Data_party_logos/sweden1_test_data/Sverige_sverigedemokraterna_YOLO"
    json_label_folder = os.path.join(main_folder, "labels")

    # Create folder for storing 
    txt_label_folder = os.path.join(main_folder, "labels_txt")
    os.makedirs(txt_label_folder, exist_ok=True)

    # Set the list of images ana labels 
    json_label_files = [filename for filename in os.listdir(json_label_folder) if not filename.startswith('.')]

    # Transform the labels
    for i, label in enumerate(json_label_files):
        !python /Users/pauor506/Documents/CDHU/June2022:Dec2022/Pilots/Youtube_Katrin/Code/Labelme2YOLO-main/labelme2yolo.py --json_dir {json_label_folder} --json_name {label}

    # Move all .txt to the proper folder and remove png generated in the step above
    txt_label_files = [filename for filename in os.listdir(json_label_folder) if filename.endswith('.txt')]
    for _, txt in enumerate(txt_label_files):
        shutil.move(os.path.join(json_label_folder,txt), os.path.join(txt_label_folder,txt))

    # Remove  png generated in the step above
    png_files = [filename for filename in os.listdir(json_label_folder) if filename.endswith('.png')]
    for _, png in enumerate(png_files):
        os.remove(os.path.join(json_label_folder,png))

# Funtion to split a folder it with images and labels into images->test/train/val and labels->test/train/val
def train_val_test_split(main_folder:str,labels_folder_name:str, val_ratio:float, test_ratio:float):

    # Get the list of image and label files
    image_folder = os.path.join(main_folder, "images")
    label_folder = os.path.join(main_folder, labels_folder_name)
    image_files = [filename for filename in os.listdir(image_folder) if not filename.startswith('.')]
    label_files = [filename for filename in os.listdir(label_folder) if not filename.startswith('.')]

    # Calculate the number of files in the validation set
    num_validation = int(len(image_files) * val_ratio)
    num_test = int(len(image_files) * test_ratio)

    # Shuffle the files randomly
    random.shuffle(image_files)

    # Split the files into training and validation sets
    val_images = image_files[:num_validation]
    test_images = image_files[num_validation:num_validation+num_test]
    train_images = image_files[num_validation+num_test:]
    train_labels = [os.path.splitext(filename)[0] + ".txt" for filename in train_images]
    val_labels = [os.path.splitext(filename)[0] + ".txt" for filename in val_images]
    test_labels = [os.path.splitext(filename)[0] + ".txt" for filename in test_images]

    # Create the training and validation folders for images and labels
    train_image_folder = os.path.join(image_folder, "train")
    val_image_folder = os.path.join(image_folder, "val")
    test_image_folder = os.path.join(image_folder, "test")
    train_label_folder = os.path.join(label_folder, "train")
    val_label_folder = os.path.join(label_folder, "val")
    test_label_folder = os.path.join(label_folder, "test")
    os.makedirs(train_image_folder, exist_ok=True)
    os.makedirs(val_image_folder, exist_ok=True)
    os.makedirs(test_image_folder, exist_ok=True) 
    os.makedirs(train_label_folder, exist_ok=True)
    os.makedirs(val_label_folder, exist_ok=True)
    os.makedirs(test_label_folder, exist_ok=True)


    # Moving files 
    for image_file, label_file in zip(train_images, train_labels):
        shutil.move(os.path.join(image_folder, image_file), os.path.join(train_image_folder, image_file))
        shutil.move(os.path.join(label_folder, label_file), os.path.join(train_label_folder, label_file))
    for image_file, label_file in zip(val_images, val_labels):
        shutil.move(os.path.join(image_folder, image_file), os.path.join(val_image_folder, image_file))
        shutil.move(os.path.join(label_folder, label_file), os.path.join(val_label_folder, label_file))
    for image_file, label_file in zip(test_images, test_labels):
        shutil.move(os.path.join(image_folder, image_file), os.path.join(test_image_folder, image_file))
        shutil.move(os.path.join(label_folder, label_file), os.path.join(test_label_folder, label_file))

In [5]:
# Function to re-label (for multiclass detection). 
# The starting folder is the labels directory 
def relabel(main_folder:str, new_label:int):
    sub_data_folders = [folder for folder in os.listdir(main_folder) if not folder.startswith('.')]
    for folder in sub_data_folders:
        # Get a list of files in the main folder
        sub_folder_path = os.path.join(main_folder,folder)
        label_files = [filename for filename in os.listdir(sub_folder_path) if not filename.startswith('.')]
        # Iterate over each file in the folder
        for filename in label_files:
            # Open the file in read-write mode
            with fileinput.FileInput(os.path.join(sub_folder_path, filename), inplace=True) as file:
                # Iterate over each line in the file
                for line in file:
                    # Split the line into a list of values
                    values = line.strip().split()
                    # Replace the first value with the new label
                    values[0] = str(new_label)
                    # Join the values back into a string
                    new_line = ' '.join(values)
                    # Print the modified line to stdout (which will be redirected to the file)
                    print(new_line)


In [9]:
# List of folders
data_dirs = ['spain/España_Izquierda_unida_partido', 'spain/España_Partido_Popular','italy/Italia_Forza_Italia_Partito','italy/Italia_Partito_del_comunisti_italiani']
for dd in data_dirs:
    # Set the main folder path
    main_folder = "/Users/pauor506/Documents/CDHU/June2022:Dec2022/Pilots/Youtube_Katrin/Data_party_logos/" + dd

    # Process json files
    jsonTOyolo(main_folder)

Converting España_Izquierda_unida_partido483.json ...
Converting España_Izquierda_unida_partido211.json ...
Converting España_Izquierda_unida_partido138.json ...
Converting España_Izquierda_unida_partido99.json ...
Converting España_Izquierda_unida_partido246.json ...
Converting España_Izquierda_unida_partido401.json ...
Converting España_Izquierda_unida_partido513.json ...
Converting España_Izquierda_unida_partido460.json ...
Converting España_Izquierda_unida_partido319.json ...
Converting España_Izquierda_unida_partido358.json ...
Converting España_Izquierda_unida_partido476.json ...
Converting España_Izquierda_unida_partido266.json ...
Converting España_Izquierda_unida_partido270.json ...
Converting España_Izquierda_unida_partido335.json ...
Converting España_Izquierda_unida_partido227.json ...
Converting España_Izquierda_unida_partido362.json ...
Converting España_Izquierda_unida_partido226.json ...
Converting España_Izquierda_unida_partido271.json ...
Converting 

In [11]:
# List of folders
data_dirs = ['norway/Norge_Arbeiderpartiet', 'norway/Norge_Høyre',
             'spain/España_Izquierda_unida_partido', 'spain/España_Partido_Popular',
             'italy/Italia_Forza_Italia_Partito','italy/Italia_Partito_del_comunisti_italiani']
for dd in data_dirs:
    # Set the main folder path
    main_folder = "/Users/pauor506/Documents/CDHU/June2022:Dec2022/Pilots/Youtube_Katrin/Data_party_logos/" + dd

    # Split
    labels_folder_name = 'labels'
    val_ratio = 0.2
    test_ratio = 0.1
    train_val_test_split(main_folder, labels_folder_name, val_ratio, test_ratio)

In [13]:
# Re-label 
#main_f = '/Users/pauor506/Documents/CDHU/June2022:Dec2022/Pilots/Youtube_Katrin/Data_party_logos/sweden1_test_data/Sverige_socialdemokraterna_YOLO_ssd_1/labels/'

labels_dirs = ['norway/Norge_Høyre','spain/España_Partido_Popular',
             'italy/Italia_Partito_del_comunisti_italiani']
for dd in labels_dirs:
    # Set the main folder path
    main_f = "/Users/pauor506/Documents/CDHU/June2022:Dec2022/Pilots/Youtube_Katrin/Data_party_logos/" + dd +'/labels/'
    # Set the new label
    label = 1
    relabel(main_f, label)